<a href="https://colab.research.google.com/github/haryobimo96/ct-denoising-redcnn/blob/master/CT_RED_CNN_Reconstructor.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# pip install pydicom

# 1. Import Libraries

In [1]:
import numpy as np
import tensorflow as tf
import pydicom as dicom
from tensorflow.keras.models import load_model
from matplotlib import pyplot

In [ ]:
import time
import psutil
import tracemalloc
import pynvml
import tensorflow as tf
import threading


pynvml.nvmlInit()
handle = pynvml.nvmlDeviceGetHandleByIndex(0)

cpu_utilization_samples = []
stop_sampling = False

def sample_cpu_utilization(interval=1.0):
    while not stop_sampling:
        cpu_utilization_samples.append(psutil.cpu_percent(interval=interval))

sampling_thread = threading.Thread(target=sample_cpu_utilization)
sampling_thread.start()

start_time = time.time()
tracemalloc.start()
process = psutil.Process()
cpu_times_start = process.cpu_times()

In [3]:
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

Num GPUs Available:  1


# 2. Extract the saved model's weight

In [4]:
# !unzip model_weight.zip -d '/content/'

In [5]:
model = tf.keras.models.load_model('C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (12 mar)/Code/mymodelpoisson_CT')

# 3. Initiate the patch denoising and reconstruction process

In [6]:
class FileManager():

    def __init__(self,directory):
        self.directory = directory

    def DicomFiles(self):
        PathDicom = self.directory
        lstFilesDCM = []
        for dirName, subdirList, fileList in os.walk(PathDicom):
            for filename in sorted(fileList):
                if ".dcm" in filename.lower():
                    lstFilesDCM.append(os.path.join(dirName,filename))
        return lstFilesDCM

    def FileCounter(self):
        file_count = sum(len(files) for _, _, files in 
                         sorted(os.walk(self.directory)))
        return file_count

In [ ]:
import os
import numpy as np
import pydicom
import matplotlib.pyplot as plt
from tensorflow.keras.models import load_model

class ImageReconstructor():
    def __init__(self, img, pixel_size, patch_size, patch_stride, pad_to_256=False):
        self.img = img
        self.pixel_size = int(round(pixel_size))
        self.patch_size = patch_size
        self.patch_stride = patch_stride
        self.pad_to_256 = pad_to_256
        
        if self.pad_to_256 and self.pixel_size < 256:
            pad_width = (256 - self.pixel_size) // 2
            self.img = np.pad(self.img, ((pad_width, pad_width), (pad_width, pad_width)), mode='reflect')
            self.pixel_size = 256

    def patch_extractor(self):
        patches_x = (self.pixel_size - self.patch_size) // self.patch_stride
        patches_y = (self.pixel_size - self.patch_size) // self.patch_stride
        patch_array = np.zeros((patches_x * patches_y, self.patch_size, self.patch_size), dtype='float32')
        
        idx = 0
        for j in range(0, self.pixel_size - self.patch_size, self.patch_stride):
            for i in range(0, self.pixel_size - self.patch_size, self.patch_stride):
                patch_array[idx] = self.img[i:i+self.patch_size, j:j+self.patch_size]
                idx += 1
        
        return patch_array

    def patch_reconstructor(self, array):
        patches_x = (self.pixel_size - self.patch_size) // self.patch_stride
        patches_y = (self.pixel_size - self.patch_size) // self.patch_stride
        tp = []
        divpatch = np.ones((self.patch_size, self.patch_size), dtype=np.float32)
        divisor = np.zeros((self.pixel_size, self.pixel_size), dtype=np.float32)
        ip = np.zeros((self.pixel_size, self.pixel_size), dtype=np.float32)

        array = array.reshape(array.shape[0], array.shape[1], array.shape[2], 1)

        for k in range(len(array)):
            x = model.predict(array[k].reshape(1, self.patch_size, self.patch_size, 1))
            tp.append(x.reshape(self.patch_size, self.patch_size))

        idx = 0
        for j in range(0, self.pixel_size - self.patch_size, self.patch_stride):
            for i in range(0, self.pixel_size - self.patch_size, self.patch_stride):
                patch = np.array(tp[idx])
                padded_patch = np.pad(patch, ((i, self.pixel_size - (i + self.patch_size)), 
                                              (j, self.pixel_size - (j + self.patch_size))), mode='constant')
                divisor_padded = np.pad(divpatch, ((i, self.pixel_size - (i + self.patch_size)),
                                                   (j, self.pixel_size - (j + self.patch_size))), mode='constant')
                ip += padded_patch
                divisor += divisor_padded
                idx += 1
        
        ipfinal = np.divide(ip, divisor, where=divisor != 0)
        return ipfinal



# 4. Read and execute the file

In [ ]:
ds = dicom.read_file('C:/Matthew/Kuliah/Semester 8/Pengolahan Data/Progress code terbaru (17 feb)/Data/Added Noise/CATHLAB/Poisson/CATHLAB Noisy (Poisson 0.70)/FCAVDP00.dcm')
img = ds.pixel_array
pyplot.imshow(img,cmap='gray')

In [ ]:
I = ImageReconstructor(img, 256, 64, 4)
It =  I.patch_extractor()

In [ ]:
imgt = I.patch_reconstructor(It)
ds.PixelData = imgt.astype('uint16').tobytes()
ds.save_as('HASIL_CATHLAB_RED-CNN_Poisson_FCAVDP00(0.70).dcm')